In [1]:
import torch
import torch.nn as nn # NN networks (CNN, RNN, losses)
import torch.optim as optim # Aptimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F # Activarions func (ReLU, Sigmoid) also included in nn
from torch.utils.data import DataLoader # Dataset manager
import torchvision.datasets as datasets # Datasets
import torchvision.transforms as transforms # Transformation to datasets
import torchvision
import sys


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparametrs
in_channels = 3
num_classes = 10
learning_rate = 0.001
num_epoches = 1
batch_size = 64

In [3]:
# Load data
train_dataset = datasets.CIFAR10(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Identity(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        
    def forward(self, x):
        return x


In [5]:
# Load the pretrained model
model = torchvision.models.vgg16(pretrained=True)
# print(model)


d:\Projects\PythonProjects\_venvs\ml_venv_p39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Projects\PythonProjects\_venvs\ml_venv_p39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# What we did?
# If we want to delete some layer of pretrained NN
# we could create NN class, that return the input -> nothing do
# We set the avgpool equal to our class, that return input 
model.avgpool = Identity()
model.classifier = nn.Linear(512, 10)
# print(model)
model.to(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [7]:
# Train model
for epoch in range(num_epoches):

    for batch_ind, (data, targets) in enumerate(train_loader):
        if batch_ind % 5 == 0:
            print(f"{batch_ind} complete")
        # Data on cude
        data = data.to(device=device) # ([64, 1, 28, 28])
        targets = targets.to(device=device)
        
        # Forawrd
        scores = model(data) # Equal to model.forward(data)
        loss = criterion(scores, targets)

        # Backprop
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()


0 complete
5 complete
10 complete
15 complete
20 complete
25 complete
30 complete
35 complete
40 complete
45 complete
50 complete
55 complete
60 complete
65 complete
70 complete
75 complete
80 complete
85 complete
90 complete
95 complete
100 complete
105 complete
110 complete
115 complete
120 complete
125 complete
130 complete
135 complete
140 complete
145 complete
150 complete
155 complete
160 complete
165 complete
170 complete
175 complete
180 complete
185 complete
190 complete
195 complete
200 complete
205 complete
210 complete
215 complete
220 complete
225 complete
230 complete
235 complete
240 complete
245 complete
250 complete
255 complete
260 complete
265 complete
270 complete
275 complete
280 complete
285 complete
290 complete
295 complete
300 complete
305 complete
310 complete
315 complete
320 complete
325 complete
330 complete
335 complete
340 complete
345 complete
350 complete
355 complete
360 complete
365 complete
370 complete
375 complete
380 complete
385 complete
390 comp

In [8]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    print(f"{float(num_correct) / float(num_samples)}")
    model.train()

In [9]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


0.62492
0.6174
